# Working with NIfTI files using Nibabel and Nilearn

This lesson shows us how we can load NIfTI files in Python so that we can use them. We'll learn to inspect header information, as well as how to access the voxel data. Finally, we'll visualize the image directly in the notebook.

## 1. Loading a NIfTI file

Python has an amazing package called [nibabel](https://nipy.org/nibabel/) that loads in all sorts of imaging formats, including the most common MRI format, NIfTI (`.nii` or `.nii.gz`). Without nibabel, neuroimaging in Python probably wouldn't exist. We'll import nibabel as it's abbreviation, `nib`, which is how nibabel is conventionally imported in programs.  

In [ ]:
import nibabel as nib

anat_file = 'data/sub-01/anat/sub-01_T1w.nii.gz'
anat_file = nib.load(anat_file)

anat_file

You can see that `anat_file` is an object (an instance of `Nifti1Image`, specifically). 

A NIfTI image consists of a header, which contains metadata, and the actual image, which is a 3- or 4-dimensional matrix. First, let's check out the header. 

## 2. NIfTI Headers

`anat_img` is a `NiftiImage` object, which has several attributes and methods. Printing the `header` attribute will show you the header data:

In [ ]:
print(anat_file.header)

You can also just show the **affine matrix** in the header. The affine matrix defines the space that the data lives in.

We can see the affine matrix by calling the `affine` attribute. You'll notice that the values correspond to the `srow_*` keys in the header:

In [ ]:
print(anat_file.affine)

## 3. NIfTI image data

The actual data in the file is stored separately as a matrix. We can access this data by calling the `get_fdata()` method, which will return the data as a `numpy` array. 

In [ ]:
img_array = anat_file.get_fdata()

img_array

For brevity, Jupyter only shows a condensed version of the array. Each number/element is a voxel, and the value corresponds to its intensity value.  

There is clearly a lot of data not being printed. To get the dimensions of the data, we can call `shape`, which will give us the number of voxels in the x, y, and z directions:

In [ ]:
print(anat_file.shape)

# or using img_array
img_array.shape

We see that it is a 3D image; it is an anatomical image that has *one* 3D volume. Meanwhile, functional images collect volumes over time. If this *was* a functional image, we would see a fourth dimension, which would correspond to the number of volumes collected.


To get a sense of how many different values are actually in the data, we can plot a histogram:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.hist(img_array.ravel(), bins=30)
plt.show()

While there are many close-to-zero voxels in there (non-brain voxels), there are many voxels with high intensity values that are brain and other tissue voxels.

## 4. Visualizing images directly in Python using Nilearn

Nilearn is a powerful package that is essential to neuroimaging in Python. It was originally developed to facilitate machine-learning task on MRI data (hence the name), but now, in addition to all sorts of analysis functions, it includes a whole bunch of handy functions to work with MRI data in general. This includes a bunch of plotting functions. 

We can plot our anatomical image using the `plot_anat` function imported from nilearn. `plot_anat` has a number of parameters you can specify, such as `cut_coords` to determine which slices of the image we will look at. 

In [ ]:
from nilearn import plotting

plotting.plot_anat(anat_file, vmax=800, cut_coords=[0, 0, 0])
plt.show()

In addition to orthographic plotting (showing a slice in x, y, and z directions), we can also plot slices along a certain axis by passing a list of coordinates to `cut_coords`, and specifying the axis in `display_mode`. Note that these coordinates are not the voxel coordinates/indices, but the actual image coordinates. 

In [ ]:
plotting.plot_anat(anat_file, cut_coords=[-40, -20, 0, 20, 40], display_mode='x', vmax=800)
plt.show()

We can also plot the image *interactively* in Jupyter by using the `view_img` function. This is a great way to quickly view a file, but I wouldn't recommend it for your primary interactive visualization tool because it's not a full-featured visualization tool. For that, I would recommend GUI's such as [FSLEyes](https://users.fmrib.ox.ac.uk/~paulmc/fsleyes/userdoc/latest/) or [MRIcron](https://www.nitrc.org/projects/mricron). Nevertheless, we *will* be using `view_img` throughout the course to quickly check some of our results.

In [ ]:
plotting.view_img(anat_file, bg_img=False, cmap='binary_r', symmetric_cmap=False, 
                  colorbar=False, vmax=800)

## 4. Excercise: Inspecting a functional image

We could have very well easily done the previous section with a functional MRI image. Nibabel will treat a functional image in the exact same way as an anatomical image, and the same methods/functions can be used. Quickly, we'll just visualize a volume from a functional image. 

In [ ]:
func_file = 'data/sub-01/func/sub-01_task-motor_run-01_bold.nii.gz'
func_file = nib.load(func_file)

Keep in mind that the functional image has 4 dimensions. We can only plot one volume at a time, so we can pick a slice to plot. We can use nilearn's `index_img` function to get a single volume from a functional image:

In [ ]:
from nilearn.image import index_img

first_volume = index_img(func_file, index=0)
plotting.plot_anat(first_volume)

I'll let you answer the following:

1. What are the dimensions/shape of the image?
2. What does the affine matrix look like?
3. What does the 5th volume look like? Because indexing starts a 0 in python, the fifth image is indexed using 4.